In [13]:
# Create dataset YAML configuration
import yaml

dataset_config = {
    'path': r'C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2',
    'train': 'train',
    'val': 'val',
    'test': 'test',
    'names': {
        0: 'OK',
        1: 'Defect',
    }
}

with open('metal_casting.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

In [14]:
#Analyze the number of images per class per dataset folder

import os

def count_images_per_class(folder_paths):
    """
    Count the number of images per class in train, val, and test folders
    Args:
        folder_paths (dict): {'train': path, 'val': path, 'test': path}
    Returns:
        dict: Number of images per class per folder
    """
    image_counts = {}
    for folder, folder_path in folder_paths.items():
        image_counts[folder] = {}
        # Loop through each class folder
        for class_name in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path):  # Ensure it's a directory
                # Count images in the class folder
                image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
                image_counts[folder][class_name] = len(image_files)
    return image_counts

base_path = dataset_config['path']
train_path = os.path.join(base_path, dataset_config['train'])
val_path = os.path.join(base_path, dataset_config['val'])
test_path = os.path.join(base_path, dataset_config['test'])

# Configuration
dataset_dirs = {
    'train': train_path,
    'val': val_path,
    'test': test_path
}

classes = list(dataset_config['names'].values())

class_image_counts  = count_images_per_class(dataset_dirs)

# Print results
print("Number of images per class per split:")
for folder, class_counts in class_image_counts.items():
    print(f"\n{folder}:")
    for class_name, count in class_counts.items():
        print(f"  {class_name}: {count} images")


Number of images per class per split:

train:
  Defect: 2966 images
  OK: 2184 images

val:
  Defect: 616 images
  OK: 476 images

test:
  Defect: 629 images
  OK: 477 images


In [15]:
# Pre-requite to run the training. Follow the following steps for Torch setup
# 1. Make sure you have a Nvidia GPU to support CUDA
# 2. Install the Torch based on the guideline from https://pytorch.org/get-started/locally/
# 3. Run the following to validate the CUDA, Torch and Torchvision version

import torch, torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")


PyTorch version: 2.6.0+cu126
Torchvision version: 0.21.0+cu126
CUDA available: True
CUDA version: 12.6


In [16]:
#Yolo model training

from ultralytics import YOLO

# Initialize model
model = YOLO('yolo11n-cls.pt')

# Start training
results = model.train(
    data=r'C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2',
    epochs=100,
    imgsz=224,
    batch=32,
    optimizer='Adam',
    seed=42,
    device='0',  # '0' for GPU, 'cpu' for CPU
    name='yolov11n_classifier'
)

New https://pypi.org/project/ultralytics/8.3.92 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.85  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2, epochs=100, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov11n_classifier6, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False

train: Scanning C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\train... 5150 images, 0 corrupt: 100%|██████████| 5150/5150 [00:02<00:00, 2013.02it/s]


train: New cache created: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\train.cache


val: Scanning C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\val... 1092 images, 0 corrupt: 100%|██████████| 1092/1092 [00:00<00:00, 1434.80it/s]


val: New cache created: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\val.cache
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\yolov11n_classifier6
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

    100/100     0.426G    0.03066         30        224: 100%|██████████| 161/161 [00:05<00:00, 31.05it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [00:00<00:00, 93.64it/s]

                   all      0.996          1



100 epochs completed in 0.164 hours.
Optimizer stripped from runs\classify\yolov11n_classifier6\weights\last.pt, 3.2MB
Optimizer stripped from runs\classify\yolov11n_classifier6\weights\best.pt, 3.2MB

Validating runs\classify\yolov11n_classifier6\weights\best.pt...
Ultralytics 8.3.85  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\train... found 5150 images in 2 classes  
val: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\val... found 1092 images in 2 classes  
test: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\test... found 1106 images in 2 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [00:00<00:00, 77.18it/s]


                   all      0.996          1
Speed: 0.0ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\yolov11n_classifier6


In [17]:
# Validate the model
metrics = model.val(data='metal_casting.yaml', split = 'test')  


Ultralytics 8.3.85  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs


test: Scanning C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\test... 1106 images, 0 corrupt: 100%|██████████| 1106/1106 [00:00<00:00, 1455.78it/s]


test: New cache created: C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\test.cache


               classes   top1_acc   top5_acc: 100%|██████████| 35/35 [00:00<00:00, 70.92it/s]


                   all      0.998          1
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\yolov11n_classifier62


In [18]:
metrics

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EC80999D90>
curves: []
curves_results: []
fitness: 0.9990958273410797
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9981916546821594, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9990958273410797}
save_dir: WindowsPath('runs/classify/yolov11n_classifier62')
speed: {'preprocess': 0.029185352706776472, 'inference': 0.2953415008381373, 'loss': 0.00020054239101007336, 'postprocess': 0.0002992765832301667}
task: 'classify'
top1: 0.9981916546821594
top5: 1.0

In [ ]:
#Run inference

from ultralytics import YOLO

# Model Path
model_path = r'C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\runs\classify\yolov11n_classifier6\weights\best.pt'

# Load a pretrained YOLO11n model
model = YOLO(model_path)

sample_image_path = r'C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting_2\test\Defect\cast_def_0_*.jpeg'

# Run inference on 'bus.jpg' with arguments
result = model.predict(sample_image_path, save = True, show_conf = True)

# View results
for r in result:
    print(r.to_df())  


image 1/592 C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting\test\0_def_front\cast_def_0_1059.jpeg: 224x224 Defect 1.00, OK 0.00, 6.1ms
image 2/592 C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting\test\0_def_front\cast_def_0_1063.jpeg: 224x224 Defect 1.00, OK 0.00, 5.6ms
image 3/592 C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting\test\0_def_front\cast_def_0_108.jpeg: 224x224 Defect 1.00, OK 0.00, 10.5ms
image 4/592 C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting\test\0_def_front\cast_def_0_1096.jpeg: 224x224 Defect 1.00, OK 0.00, 8.8ms
image 5/592 C:\Users\User\OneDrive\Documents\Programming\metal_cast_detection\notebooks\model_training\datasets\metal_casting\test\0_def_front\cast_def_0_112.jpeg: 224x224 Defect 1.00, OK 0.00, 5